# Are We Rich Yet?

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
def calculate_commission(price, quantity):
    #fix rate pricing from IB including all exchange and regularotry fees
    #https://www.interactivebrokers.com/en/index.php?f=1590&p=stocks1&ns=T
    return np.minimum(np.maximum(0.005*quantity, 1), price*quantity*0.005)

'''
input:
signal_column: name of the column that indicates the signal,  1 for long signal or -1 for short signal
price_column: name of the column responding to order execution price, could be open or close
include_commision: deduct commission for each transaction if True, default to True

output: 
balance, profit, ROI

NOTE: this is LONG only evaluation, SHORT is not implemented yet. SHORT position will involve margin, so we need to decide 
later.

'''
# Long only
def evaluate_profit(df, intitial_balance, signal_column, price_column, include_commission=True):

    balance = intitial_balance
    current_position = 0 #1 for long position, -1 for short, 0 for closed position or no position
    pos_quantity = 0 # current position quantity
    pos_price = 0 #current position price

    #iterate from first to n-1 row, as we want to close our position on the last day.
    for index, row in df.head(-1).iterrows():
        new_position = row[signal_column]
        current_price = row[price_column]
        
        # receive signal to reverse current position
        if current_position != new_position:
            # bullish market: close short position and open a long position
            if new_position == 1:
                #open a long position (short position not implemented yet)
                pos_quantity = np.floor_divide(balance, current_price) 
                balance = balance - current_price * pos_quantity
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)
                
                pos_price = current_price
                current_position = new_position
                #print "long ", row['date'], balance, pos_quantity, pos_price
            
            # bearish market: close long position and open a short position
            else:
                #close the long position (short position not implemented yet)
                balance = balance + current_price * pos_quantity
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)
                    
                pos_quantity = 0
                pos_price = 0
                current_position = new_position
                #print "close", row['date'], balance, pos_quantity, pos_price
            
        #hold position and do nothing
        else:
            #print "hold ", row['date'], balance, pos_quantity, pos_price
            pass
            
            
    #found open position on the last day, let's close it
    if pos_quantity != 0:
        # get price of last day
        current_price = df.tail(1).iloc[0][price_column]
        balance = balance + pos_quantity * current_price
        if include_commission:
            balance = balance - calculate_commission(current_price, pos_quantity)
    
    profit = balance - intitial_balance
    ROI = profit/intitial_balance
    
    return  balance, profit, ROI

# Long and Short
def evaluate_profit2(df, intitial_balance, signal_column, price_column, include_commission=True):

    balance = intitial_balance
    current_position = -1 #1 for long position, 0 for short
    pos_quantity = 0 # current position quantity
    pos_price = 0 #current position price

    #iterate from first to n-1 row, as we want to close our position on the last day.
    for index, row in df.head(-1).iterrows():
        new_position = row[signal_column]
        current_price = row[price_column]
        
        # receive signal to reverse current position
        if current_position != new_position:
            # bullish market: close short position and open a long position
            if new_position == 1:
                #close the short position
                balance = balance + pos_quantity * (2 * pos_price - current_price)
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)

                #open a long position 
                pos_quantity = np.floor_divide(balance, current_price) 
                balance = balance - current_price * pos_quantity
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)
                
                pos_price = current_price
                current_position = new_position
                #print "long ", row['date'], balance, pos_quantity, pos_price
            
            # bearish market: close long position and open a short position
            else:
                #close the long position
                balance = balance + current_price * pos_quantity
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)
                
                #open a short position
                pos_quantity = np.floor_divide(balance, current_price) 
                balance = balance - current_price * pos_quantity
                pos_price = current_price
                current_position = new_position
                #print "close", row['date'], balance, pos_quantity, pos_price
            
        #hold position and do nothing
        else:
            #print "hold ", row['date'], balance, pos_quantity, pos_price
            pass
            
            
    #found open position on the last day, let's close it
    if pos_quantity != 0:
        # get price of last day
        current_price = df.tail(1).iloc[0][price_column]
        
        #close long position
        if current_position == 1:
            balance = balance + pos_quantity * current_price
            if include_commission:
                balance = balance - calculate_commission(current_price, pos_quantity)
        
        #close short position
        else:
            balance = balance + pos_quantity * (2 * pos_price - current_price)
            if include_commission:
                balance = balance - calculate_commission(current_price, pos_quantity)
            
    profit = balance - intitial_balance
    ROI = profit/intitial_balance
    
    return  balance, profit, ROI

### Get data

In [3]:
df=pd.read_csv("data/IYZ.csv")
df.head()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
0,2010-01-04,20.370001,20.549999,20.270000,20.549999,664200,20.680000,True,0.006326,18.719999,False,-0.089051,0,20.545079,19.9480,19.350920,1.004120,5.986360,False,True,False,0,19.245605,18.466066,18.432535,1,1,19.030677,18.358833,18.371497,1,1,19.822958,1,34.171492,33.666344,15.692957,True,True,1,96,1,122.917840,1,0.393756,0.403865,-0.010110,0,2.048084,1,79.657038,0,4.900454,0,68.259660,0,56.239960,0,-0.000000,0,694259.549529,1,49.166583,52.670801,0,81.338670,27.112890,0,0.164700,1,20.456666,22.773333,20.363333,22.866666,18.046666,25.183333,17.953333,0
1,2010-01-05,20.610001,20.760000,20.459999,20.680000,1214800,20.340000,False,-0.016441,18.900000,False,-0.086074,0,20.632786,20.0125,19.392214,1.038058,6.198986,False,True,False,0,19.301856,18.495389,18.454898,1,1,19.120800,18.397050,18.419909,1,1,19.881121,1,34.807420,36.435763,14.496814,True,True,1,100,1,149.707718,1,0.410108,0.405114,0.004994,1,2.058941,1,81.099707,0,4.444450,0,69.920669,0,60.808637,0,-6.666661,0,828264.024156,1,63.063063,50.743160,0,100.000000,60.446223,0,0.170615,1,20.633333,23.106666,20.506666,23.233333,18.033333,25.706666,17.906666,0
2,2010-01-06,20.719999,20.730000,20.270000,20.340000,804600,20.260000,False,-0.003933,18.600000,False,-0.085546,0,20.667400,20.0395,19.411600,0.739290,6.266626,False,False,False,0,19.342568,18.519821,18.473655,1,1,19.183789,18.428070,18.450120,1,1,19.969009,1,34.451424,32.362519,17.493670,True,True,1,96,1,85.708389,1,0.391124,0.402316,-0.011192,0,2.041574,1,76.927311,0,1.345291,0,60.938720,0,54.881580,0,-34.999971,0,629694.026402,1,77.477477,63.235708,0,10.627286,63.988652,1,0.175967,1,20.446667,22.583334,20.133333,22.896668,17.996666,25.033335,17.683332,0
3,2010-01-07,20.340000,20.379999,20.160000,20.260000,471900,20.110001,False,-0.007404,18.570000,False,-0.083416,0,20.677310,20.0705,19.463690,0.656144,6.046787,False,False,False,0,19.378545,18.542870,18.491429,1,1,19.248942,18.455162,18.480566,1,1,20.048108,1,33.614615,30.600584,19.263447,True,True,1,92,1,24.268894,1,0.365411,0.394935,-0.029524,0,1.902912,1,72.638398,0,-0.246179,0,59.017652,0,53.230087,0,-41.666632,0,477566.554469,1,54.954955,65.165165,0,0.000000,36.875762,1,0.180626,1,20.266666,22.223334,19.773333,22.716667,17.816665,24.673335,17.323332,1
4,2010-01-08,20.250000,20.250000,20.020000,20.110001,549100,20.150000,True,0.001989,18.400000,False,-0.085032,0,20.670872,20.0925,19.514128,0.515130,5.757097,False,False,False,0,19.407230,18.563627,18.507535,1,1,19.296747,18.478643,18.507601,1,1,20.760000,0,32.227258,28.760971,21.612208,True,True,1,88,1,-45.787970,1,0.329136,0.381775,-0.052639,0,1.685848,1,70.597898,0,-0.049702,0,55.485710,0,53.570511,0,-54.621765,0,333937.741315,1,29.279324,53.903919,0,0.000000,3.542429,1,0.184391,1,20.126667,21.943335,19.493334,22.576668,17.676666,24.393336,17.043333,0


In [4]:
df.tail()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
1478,2015-11-16,28.900000,29.309999,28.830000,29.309999,84100,29.389999,True,0.002729,27.940001,False,-0.046742,1,30.531382,29.6840,28.836617,0.279320,5.709355,False,False,False,0,29.076328,29.204370,29.298876,1,0,28.976103,29.241528,29.428410,1,0,30.067442,0,18.896958,22.420464,24.627827,False,False,1,60,1,-121.505885,1,0.162721,0.317523,-0.154802,0,1.148712,1,57.662795,0,-3.585530,0,50.035753,1,50.772526,0,-72.571486,1,-76270.418341,0,20.900071,10.839091,1,66.985684,22.328561,1,0.053348,1,29.149999,30.569999,27.719999,31.999999,26.299999,33.419999,24.869999,0
1479,2015-11-17,29.350000,29.590000,29.200001,29.389999,209900,29.780001,True,0.013270,27.000000,False,-0.081320,1,30.528164,29.6890,28.849835,0.321846,5.653034,False,False,False,0,29.088629,29.206829,29.299783,1,0,28.979545,29.242787,29.428221,1,0,29.894200,0,17.928899,25.628114,23.027874,False,True,1,60,1,-61.672994,1,0.143043,0.282627,-0.139584,0,0.874122,1,56.797205,0,-3.195000,0,51.129096,1,44.416942,0,-68.000057,1,-73410.275764,0,37.904052,21.177492,1,100.000000,55.661895,0,0.055936,1,29.393333,30.956667,28.206667,32.143333,26.643333,33.706667,25.456667,0
1480,2015-11-18,29.490000,29.840000,29.240000,29.780001,1379200,29.780001,False,0.000000,27.120001,False,-0.089322,1,30.499962,29.7300,28.960038,0.532470,5.179696,False,False,False,0,29.115741,29.214420,29.304561,1,0,28.990811,29.246183,29.429696,1,0,28.830000,1,17.600590,27.186669,20.790128,False,True,1,60,1,-16.082587,1,0.157107,0.257523,-0.100416,0,0.489928,1,64.333933,0,-1.259944,0,56.165021,1,53.357393,0,-45.714229,1,285503.997692,1,69.257220,42.687115,1,100.000000,88.995228,0,0.058436,1,29.620000,31.270002,28.660001,32.230001,27.009999,33.880003,26.050000,0
1481,2015-11-19,29.799999,29.860001,29.700001,29.780001,180900,29.680000,False,-0.003358,26.430000,False,-0.112492,1,30.507002,29.7395,28.971997,0.526385,5.161502,False,False,False,0,29.141791,29.221912,29.309292,1,0,29.003276,29.250148,29.431356,1,0,28.850200,1,17.340420,26.787955,20.225693,False,True,1,56,1,15.603011,1,0.166336,0.239286,-0.072950,0,0.093857,1,67.438783,0,-0.733330,0,56.165021,1,54.729219,0,-45.714229,1,406377.655225,1,82.610291,63.257188,0,100.000000,100.000000,0,0.060819,1,29.780001,31.590003,28.980002,32.390002,27.170000,34.200004,26.370001,0
1482,2015-11-20,29.860001,29.900000,29.639999,29.680000,153400,29.170000,False,-0.017183,25.219999,False,-0.150270,0,30.489814,29.7270,28.964186,0.469193,5.132129,False,False,False,0,29.162897,29.227979,29.312981,1,0,29.013116,29.253619,29.432630,1,0,28.890592,1,16.913323,25.572952,20.355045,False,True,1,52,1,17.310571,1,0.163693,0.224167,-0.060474,0,-0.167344,0,58.479730,0,-1.066667,0,54.495110,1,54.499672,0,-51.428571,1,385091.955069,1,88.577258,80.148257,0,72.755132,90.918377,0,0.062966,1,29.740000,31.510000,28.899999,32.350001,27.129999,34.120001,26.289998,0


In [5]:
dftouse=df.copy()

### Feature Engineering

In [6]:
IGNORE = ['date', 'result_1','close_1','perf_1','result_14','close_14','perf_14','results']

In [7]:
INDICATORS=[]
for v in df.columns:
    l=df[v].unique()
    if len(l) <= 10 and v not in IGNORE:
        print v, l
        INDICATORS.append(v)

bb_squeeze [False True]
bb_signalup [True False]
bb_signaldn [False True]
bb_signal [ 0.  1.]
ema_signal1 [1 0]
ema_signal2 [1 0]
kama_signal1 [1 0]
kama_signal2 [1 0]
sar_signal [1 0]
adx_trend [True False]
adx_direction [True False]
adx_signal [ 1.  0.]
aroon_signal [1 0]
cci_signal [ 1.  0.]
macd_signal [0 1]
ppo_signal [1 0]
mfi_signal [ 0.  1.]
roc_signal [ 0.  1.]
rsi_signal [ 0.  1.]
ult_signal [ 0.  1.]
wr_signal [ 0.  1.]
ad_signal [1 0]
sslow_signal [ 0.  1.]
srsi_signal [ 0.  1.]
trix_signal [1 0]
cv_signal [0 1]


In [8]:
STANDARDIZABLE = []
for v in df.columns:
    if v not in INDICATORS and v not in IGNORE:
        print v
        STANDARDIZABLE.append(v)

open
high
low
close
volume
bb_upper
bb_middle
bb_lower
bb_pct
bb_bandwidth
ema50
ema150
ema200
kama50
kama150
kama200
sar
adx
plus_di
minus_di
aroon_osc
cci
macd
macd_sigline
macd_hist
ppo
mfi
roc
rsi
ult_osc
willr
ad_osc
stoch_slowk
stoch_slowd
stoch_fastk
stoch_fastd
trix
sr_pivotpts
sr_res1
sr_sup1
sr_res2
sr_sup2
sr_res3
sr_sup3


In [9]:
# from sklearn.cross_validation import train_test_split
# itrain, itest = train_test_split(xrange(dftouse.shape[0]), train_size=0.7)
# mask=np.ones(dftouse.shape[0], dtype='int')
# mask[itrain]=1
# mask[itest]=0
# mask = (mask==1)
# mask.shape, mask.sum()

In [10]:
dftouse['date'] = pd.to_datetime(dftouse['date'])
mask = (dftouse.date < '2015-01-01').values
mask.shape, mask.sum()

((1483,), 1258)

### Check ROI of signals, alone:

In [11]:
print "ROI baseline: 1.12%"
print 'ROI "result" buy-only: 67.45%'
print 'ROI "result" buy-sell: 172.49%'
    
def signalperf(signal):
    ypred = df[signal]
    df_pred = df[~mask].reset_index(drop=True)
    df_pred['pred_result'] = ypred
    df_pred['result_baseline'] = np.ones(df_pred.shape[0])

    balance, profit, ROI2 = evaluate_profit(df_pred, 10000, 'pred_result', 'close', False)
    #print 'ROI "pred" buy-only: {0:.2f}%'.format(ROI*100)

    balance, profit, ROI = evaluate_profit2(df_pred, 10000, 'pred_result', 'close', False)
    print signal + ': ROI "pred" buy-only: {0:.2f}%'.format(ROI2*100), 'buy-sell: {0:.2f}%'.format(ROI*100)
    return

ROI baseline: 1.12%
ROI "result" buy-only: 67.45%
ROI "result" buy-sell: 172.49%


In [12]:
for v in INDICATORS:
    signalperf(v)

bb_squeeze: ROI "pred" buy-only: 0.83% buy-sell: -0.48%
bb_signalup: ROI "pred" buy-only: -9.98% buy-sell: -20.92%
bb_signaldn: ROI "pred" buy-only: -1.77% buy-sell: -4.85%
bb_signal: ROI "pred" buy-only: -2.36% buy-sell: -5.85%
ema_signal1: ROI "pred" buy-only: 0.13% buy-sell: -1.25%
ema_signal2: ROI "pred" buy-only: 1.12% buy-sell: 1.12%
kama_signal1: ROI "pred" buy-only: 1.70% buy-sell: 1.94%
kama_signal2: ROI "pred" buy-only: 1.12% buy-sell: 1.12%
sar_signal: ROI "pred" buy-only: -9.83% buy-sell: -20.27%
adx_trend: ROI "pred" buy-only: 12.65% buy-sell: 24.86%
adx_direction: ROI "pred" buy-only: 1.36% buy-sell: 0.40%
adx_signal: ROI "pred" buy-only: -1.32% buy-sell: -4.57%
aroon_signal: ROI "pred" buy-only: -2.54% buy-sell: -6.78%
cci_signal: ROI "pred" buy-only: -4.73% buy-sell: -10.57%
macd_signal: ROI "pred" buy-only: -3.20% buy-sell: -9.29%
ppo_signal: ROI "pred" buy-only: -5.35% buy-sell: -11.78%
mfi_signal: ROI "pred" buy-only: -3.39% buy-sell: -7.87%
roc_signal: ROI "pred" bu

#### 1.2 Standardize the data

Use the mask to compute the training and test parts of the dataframe. Use `StandardScaler` from `sklearn.preprocessing` to "fit" the columns in `STANDARDIZABLE` on the training set. Then use the resultant estimator to transform both the training and the test parts of each of the columns in the dataframe, replacing the old unstandardized values in the `STANDARDIZABLE` columns of `dftouse` by the new standardized ones.

In [13]:
#your code here
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(dftouse[mask][STANDARDIZABLE])
dftouse[STANDARDIZABLE] = scaler.transform(dftouse[STANDARDIZABLE])
dftouse.head()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
0,2010-01-04,-1.169181,-1.157299,-1.15216,-1.118360,0.639008,20.680000,True,0.006326,18.719999,False,-0.089051,0,-1.321332,-1.279148,-1.221656,1.341702,-0.221542,False,True,False,0,-1.460919,-1.608066,-1.592815,1,1,-1.591111,-1.688776,-1.698226,1,1,-1.257753,1,1.219553,1.264466,-1.068669,True,True,1,1.313898,1,1.004782,1,1.390227,1.549988,-0.096380,0,1.164215,1,1.561731,0,1.442156,0,1.356830,0,0.191720,0,1.334246,0,1.564001,1,-0.269844,-0.148049,0,0.698039,-0.732738,0,1.451422,1,-1.142985,-0.827349,-0.890331,-1.051099,-1.175919,-0.728905,-0.909411,0
1,2010-01-05,-1.101747,-1.098349,-1.09877,-1.081848,2.142760,20.340000,False,-0.016441,18.900000,False,-0.086074,0,-1.296796,-1.260871,-1.209938,1.443046,-0.155418,False,True,False,0,-1.444752,-1.599330,-1.585983,1,1,-1.564010,-1.677462,-1.683838,1,1,-1.241792,1,1.295293,1.656456,-1.216766,True,True,1,1.378875,1,1.253799,1,1.457677,1.555591,0.078078,1,1.171118,1,1.645223,0,1.297461,0,1.505069,0,0.626544,0,1.120493,0,1.930265,1,0.257237,-0.227046,0,1.139066,0.256536,0,1.519308,1,-1.093350,-0.737175,-0.851602,-0.950513,-1.179561,-0.593715,-0.921427,0
2,2010-01-06,-1.070840,-1.106770,-1.15216,-1.177339,1.022456,20.260000,False,-0.003933,18.600000,False,-0.085546,0,-1.287113,-1.253220,-1.204436,0.550893,-0.134384,False,False,False,0,-1.433052,-1.592050,-1.580253,1,1,-1.545068,-1.668280,-1.674858,1,1,-1.217674,1,1.252893,1.079920,-0.845719,True,True,1,1.313898,1,0.658914,1,1.379371,1.543035,-0.108883,0,1.160075,1,1.403751,0,0.314070,0,0.703464,0,0.062437,0,0.212042,0,1.387529,1,0.803962,0.284918,0,-0.973098,0.361669,1,1.580734,1,-1.145794,-0.878748,-0.952477,-1.042869,-1.189577,-0.767654,-0.978934,0
3,2010-01-07,-1.177611,-1.205021,-1.18307,-1.199808,0.113814,20.110001,False,-0.007404,18.570000,False,-0.083416,0,-1.284341,-1.244436,-1.189654,0.302612,-0.202750,False,False,False,0,-1.422711,-1.585183,-1.574823,1,1,-1.525476,-1.660259,-1.665809,1,1,-1.195968,1,1.153228,0.830531,-0.626599,True,True,1,1.248921,1,0.087822,1,1.273312,1.509911,-0.320626,0,1.071905,1,1.155536,0,-0.190918,0,0.532015,0,-0.094744,0,-0.001712,0,0.971730,1,-0.050296,0.363990,0,-1.224255,-0.442993,1,1.634198,1,-1.196366,-0.976136,-1.049750,-1.092248,-1.238745,-0.860651,-1.071632,1
4,2010-01-08,-1.202899,-1.241514,-1.22241,-1.241936,0.324656,20.150000,True,0.001989,18.400000,False,-0.085032,0,-1.286142,-1.238202,-1.175341,-0.118472,-0.292838,False,False,False,0,-1.414467,-1.578999,-1.569903,1,1,-1.511101,-1.653308,-1.657774,1,1,-1.000611,0,0.987992,0.570148,-0.335794,True,True,1,1.183944,1,-0.563369,1,1.123685,1.450854,-0.587622,0,0.933880,1,1.037445,0,-0.128574,0,0.216803,0,-0.062344,0,-0.417092,0,0.579160,1,-1.024149,-0.097513,0,-1.224255,-1.432267,1,1.677407,1,-1.235698,-1.051881,-1.125406,-1.130653,-1.276986,-0.932983,-1.143730,0


We create a list `lcols` of the columns we will use in our classifier. This list should not contain the response `RESP`. How many features do we have?

In [14]:
#lcols=list(dftouse.columns)
#lcols.remove(u'results')
lcols=[]
#lcols.append('cv_signal')
for c in list(dftouse.columns):
    #if c not in INDICATORS and c not in IGNORE:
    #if c in INDICATORS:
    if c not in IGNORE:  #Original
        lcols.append(c)
print len(lcols)

70


### EDA for the data

We create a variable `ccols` which contains all variables not in our indicators list

In [15]:
ccols=[]
for c in lcols:
    if c not in INDICATORS and c not in IGNORE:
        ccols.append(c)
print len(ccols), len(INDICATORS)
ccols

44 26


['open',
 'high',
 'low',
 'close',
 'volume',
 'bb_upper',
 'bb_middle',
 'bb_lower',
 'bb_pct',
 'bb_bandwidth',
 'ema50',
 'ema150',
 'ema200',
 'kama50',
 'kama150',
 'kama200',
 'sar',
 'adx',
 'plus_di',
 'minus_di',
 'aroon_osc',
 'cci',
 'macd',
 'macd_sigline',
 'macd_hist',
 'ppo',
 'mfi',
 'roc',
 'rsi',
 'ult_osc',
 'willr',
 'ad_osc',
 'stoch_slowk',
 'stoch_slowd',
 'stoch_fastk',
 'stoch_fastd',
 'trix',
 'sr_pivotpts',
 'sr_res1',
 'sr_sup1',
 'sr_res2',
 'sr_sup2',
 'sr_res3',
 'sr_sup3']

#### 1.4 Train a SVM on this data.

In [16]:
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV

In [17]:
"""
Function
--------
cv_optimize

Inputs
------
clf : an instance of a scikit-learn classifier
parameters: a parameter grid dictionary thats passed to GridSearchCV (see above)
X: a samples-features matrix in the scikit-learn style
y: the response vectors of 1s and 0s (+ives and -ives)
n_folds: the number of cross-validation folds (default 5)
score_func: a score function we might want to pass (default python None)
   
Returns
-------
The best estimator from the GridSearchCV, after the GridSearchCV has been used to
fit the model.
     
Notes
-----
see do_classify and the code below for an example of how this is used
"""
#your code here
def cv_optimize(clf, parameters, X, y, n_folds, score_func):
    fitmodel = GridSearchCV(clf, param_grid=parameters, cv=n_folds, scoring=score_func)
    fitmodel.fit(X, y)
    return fitmodel.best_estimator_

In [18]:
from sklearn.metrics import confusion_matrix
def do_classify(clf, parameters, indf, featurenames, targetname, target1val, mask=None, reuse_split=None, score_func=None, n_folds=5):
    subdf=indf[featurenames]
    X=subdf.values
    y=(indf[targetname].values==target1val)*1
    if mask !=None:
        print "using mask"
        Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
    if reuse_split !=None:
        print "using reuse split"
        Xtrain, Xtest, ytrain, ytest = reuse_split['Xtrain'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['ytest']
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print "############# based on standard predict ################"
    print "Accuracy on training data: %0.2f" % (training_accuracy)
    print "Accuracy on test data:     %0.2f" % (test_accuracy)
    print confusion_matrix(ytest, clf.predict(Xtest))
    print "########################################################"
    return clf, Xtrain, ytrain, Xtest, ytest

In [19]:
%%time
clfsvm, Xtrain, ytrain, Xtest, ytest = do_classify(LinearSVC(loss="hinge"), {"C": [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]}, dftouse, lcols, u'result_14',1, mask=mask)

using mask
############# based on standard predict ################
Accuracy on training data: 0.60
Accuracy on test data:     0.58
[[122   1]
 [ 94   8]]
########################################################
CPU times: user 1.73 s, sys: 22.6 ms, total: 1.75 s
Wall time: 2.34 s


/Users/ela/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [20]:
clfsvm

LinearSVC(C=1e-05, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0)

In [21]:
#your code here

def evaluate(clf):

    ypred = clf.predict(Xtest)
    df_pred = df[~mask].reset_index(drop=True)
    df_pred['pred_result'] = ypred
    df_pred['result_baseline'] = np.ones(df_pred.shape[0])
    print "accuracy on test set: {0:.3f}".format((df_pred.result_14 == df_pred.pred_result).sum()/float(len(df_pred)))
    
    balance, profit, ROI = evaluate_profit2(df_pred, 10000, 'result_baseline', 'close', False)
    print "ROI baseline: {0:.2f}%".format(ROI*100)
    
    balance, profit, ROI = evaluate_profit(df_pred, 10000, 'result_14', 'close', False)
    print 'ROI "result" buy-only: {0:.2f}%'.format(ROI*100)
    
    balance, profit, ROI = evaluate_profit2(df_pred, 10000, 'result_14', 'close', False)
    print 'ROI "result" buy-sell: {0:.2f}%'.format(ROI*100)
    
    balance, profit, ROI = evaluate_profit(df_pred, 10000, 'pred_result', 'close', False)
    print 'ROI "pred" buy-only: {0:.2f}%'.format(ROI*100)
    
    balance, profit, ROI = evaluate_profit2(df_pred, 10000, 'pred_result', 'close', False)
    print 'ROI "pred" buy-sell: {0:.2f}%'.format(ROI*100)
    
evaluate(clfsvm)

accuracy on test set: 0.578
ROI baseline: 1.12%
ROI "result" buy-only: 32.48%
ROI "result" buy-sell: 72.07%
ROI "pred" buy-only: 5.70%
ROI "pred" buy-sell: 9.57%


The results obtained ought to be very similar to the efforts you put in earlier. If not its likely you wrote `cv_optimize` wrong. (Remember that we are using the same mask).

We'll reuse the training and test sets you computed above later in the homework. We do this by putting them into a dictionary `reuse_split`

In [22]:
reuse_split=dict(Xtrain=Xtrain, Xtest=Xtest, ytrain=ytrain, ytest=ytest)

## 2. Estimate costs and benefits from assumptions and data

### Our data is highly asymmetric

First notice that our data set is very highly asymmetric, with positive `RESP`onses only making up 16-17% of the samples.

In [23]:
print "whole data set", dftouse['result_14'].mean()#Highly asymmetric
print "training set", dftouse['result_14'][mask].mean(), "test set", dftouse['result_14'][~mask].mean()

whole data set 0.551584625759
training set 0.569157392687 test set 0.453333333333


This means that a classifier which predicts that EVERY customer is a negative has an accuracy rate of 83-84%. By this we mean that **a classifier that predicts that no customer will respond to our mailing** has an accuracy of 83-84%!

In [24]:
#your code here
from sklearn.linear_model import LogisticRegression
clflog = LogisticRegression(penalty="l1")
parameters = {"C": [0.001, 0.01, 0.1, 1, 10, 100]}
clflog, Xtrain, ytrain, Xtest, ytest = do_classify(clflog, parameters, dftouse, lcols, u'result_14',1, mask=mask, reuse_split=reuse_split)

using mask
using reuse split
############# based on standard predict ################
Accuracy on training data: 0.57
Accuracy on test data:     0.59
[[105  18]
 [ 75  27]]
########################################################


/Users/ela/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [25]:
clflog

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [26]:
evaluate(clflog)

accuracy on test set: 0.587
ROI baseline: 1.12%
ROI "result" buy-only: 32.48%
ROI "result" buy-sell: 72.07%
ROI "pred" buy-only: 10.47%
ROI "pred" buy-sell: 19.09%


In [27]:
from sklearn.ensemble import RandomForestClassifier
clfraf = RandomForestClassifier(n_jobs=2)
parameters = {}#{"C": [0.001, 0.01, 0.1, 1, 10, 100]}
clfraf, Xtrain, ytrain, Xtest, ytest = do_classify(clfraf, parameters, dftouse, lcols, u'result_14',1, mask=mask, reuse_split=reuse_split)

using mask
using reuse split
############# based on standard predict ################
Accuracy on training data: 1.00
Accuracy on test data:     0.60
[[92 31]
 [60 42]]
########################################################


/Users/ela/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [28]:
evaluate(clfraf)

accuracy on test set: 0.596
ROI baseline: 1.12%
ROI "result" buy-only: 32.48%
ROI "result" buy-sell: 72.07%
ROI "pred" buy-only: 8.73%
ROI "pred" buy-sell: 16.69%


## 4. Trying to improve the SVM: Feature Selection and Data Balancing

If you did the previous section right, you will find that the logistic regression model provides a better profit over some section of the profit curve than the baseline "send to everyone" classifier, while the SVM classifier is generally poor. At this might we might want to try all kinds of classifiers: from perceptrons to random forests. In the interest of time, and to study the SVM in some more detail, we'll restrict ourselves to trying to improve the SVM performance here. In real life you would try other classifiers as well.

 We wont be exhaustive in this improvement process either(which is something you should do on your project) in the interests of time, but we'll explore if feature-selection on the  SVM, and data balancing on the SVM (SVM's are known to perform better on balanced data) help.
 
( An aside: many classifiers such as SVM and decision trees struggle in their techniques on imbalanced data. You can read more at: see Weiss, Gary M., and Foster Provost. "The effect of class distribution on classifier learning: an empirical study." Rutgers Univ (2001). Also see http://pages.stern.nyu.edu/~fprovost/Papers/skew.PDF and http://www.cs.ox.ac.uk/people/vasile.palade/papers/Class-Imbalance-SVM.pdf for multiple ways to deal with the imbalance problem: balancing is not always the best option. `Sklearn` also provides a class weighting strategy: http://scikit-learn.org/stable/modules/svm.html#unbalanced-problems ). 

### Feature Selection

The Lasso, for example, implements internally, a form of feature selection by setting many coefficients to zero. Let us find coefficients that are non-zero.

#### Non zero lasso features

We write a function `nonzero_lasso` which takes the fit classifier `clfloglasso` as an argument, and spits out a dataframe of coefficients, sorted by the absolute magnitude of the coefficients. This way we can see which features dominated the logistic regression.

In [29]:
def nonzero_lasso(clf):
    featuremask=(clf.coef_ !=0.0)[0]
    return pd.DataFrame(dict(feature=lcols, coef=clf.coef_[0], abscoef=np.abs(clf.coef_[0])))[featuremask].sort('abscoef', ascending=False)

In [30]:
lasso_importances=nonzero_lasso(clflog)
lasso_importances.set_index("feature", inplace=True)
lasso_importances.head(10)

,abscoef,coef
feature,,
macd_sigline,0.086251,-0.086251
sr_res3,0.081607,-0.081607


#### 4.1 Feature importance using correlations

We can also get a notion of which features are important in the classification process by seeing how they correlate with the response. Implement some code to obtain the Pearson correlation coefficient between each of our features and the response. Do this on the training set only! Create a dataframe indexed by the features, which has columns `abscorr` the absolute value of the correlation and `corr` the value of the correlation. Sort the dataframe by `abscorr`, highest first, and show the top 25 features with the highest absolute correlation. Is there much overlap with the feature selection performed by the LASSO?

In [31]:
from scipy.stats.stats import pearsonr
correlations=[]
dftousetrain=dftouse[mask]
for col in lcols:
    r=pearsonr(dftousetrain[col], dftousetrain['result_14'])[0]
    correlations.append(dict(feature=col,corr=r, abscorr=np.abs(r)))

bpdf=pd.DataFrame(correlations).sort('abscorr', ascending=False)
bpdf.set_index(['feature'], inplace=True)
bpdf.head(25)

,abscorr,corr
feature,,
macd_sigline,0.210943,-0.210943
sr_res3,0.209055,-0.209055
sr_res1,0.208616,-0.208616
macd,0.208234,-0.208234
mfi_signal,0.205098,0.205098
sr_sup1,0.198074,-0.198074
sr_res2,0.181206,-0.181206
sr_sup3,0.178948,-0.178948
close,0.175370,-0.175370


#### Why Feature Select?

One of the reasons feature selection is done, automatically or otherwise, is that there might be strong correlations between features. Also recall polynomial regression: a large number of features can lead to overfitting. Feature selection helps curb the problem of the curse of dimensionality, where centrality measures often used in statistics go wonky at higher dimensions. Between feature-engineering which we did some of, earlier, and feature selection, is where a lot of smarts and domain knowledge comes in. You will gain this with experience.

### Create a pipeline to feature-select, standardize and train!

We shall use sklearn pipelines to do correlation-with-response based feature selection for our SVM model. Maybe such feature-selection will improve the abysmal performance. 

This does not reduce the collinearity amongst the features, for which one either needs PCA, ICA, or some feature selection using the forward-backward algorithm. We do not have the time to approach it here. 

Its very important to do response based feature selection in the right way. If you remember, we separately standardized the training and test sets. This was to prevent **any** information about the overall mean and standard deviation leaking into the test set. 

But we played a bit loose with the rules there. We standardized on the entire training set. Instead we should have been standardizing separately in each cross-validation fold. There the original training set would be broken up into a sub-training and validation set, the standardization needed to be done on those separately. This can be implemented with `sklearn` pipelines.

Such kind of "data snooping" is relatively benign though, as it used no information about the response variable. But if you do any feature selection which uses the response variable, such as choosing the "k" most correlated variables from above, its not benign any more. This is because you have leaked the response from the validation into your sub-training set, and cannot thus be confident about your predictions: you might overfit. In such a situation, you must do the feature selection inside the cross-validation fold. See http://nbviewer.ipython.org/github/cs109/content/blob/master/lec_10_cross_val.ipynb from the 2013 course for a particularly dastardly case of this, where you see that the problem is particularly exacerbated when you have many more features than samples.

Lets do this here using sklearn pipelines.

In [32]:
from sklearn import feature_selection
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest

Lets define a scorer which returns the absolute values of the pearson correlation between the feature and the response for each sample. The specific form of the scorer is dictated to us in the API docs for `SelectKBest`, see [here](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html): the first argument must be an array of scores, and the second an array of p-values.

In [33]:
def pearson_scorer(X,y):
    rs=np.zeros(X.shape[1])
    pvals=np.zeros(X.shape[1])
    i=0
    for v in X.T:
        rs[i], pvals[i]=pearsonr(v, y)
        i=i+1
    return np.abs(rs), pvals    

Lets apply the feature selection to a model which did not have any automatic feature selection and performed rather poorly before: the linear SVM. 

The `Pipeline` feature of sklearn chains various parts of a machine learning algorithm together. In this case we want to chain feature-selection and training in such a way that both happen freshly for each cross-validation fold (we wont bother to standardize in each cross-validation fold separately here for brevity, although you might want to do this).
We use the `SelectKBest` meta estimator to select the 25 most correlated/anti-correlated features. We create an instance of this meta-estimator, `selectorlinearsvm`. We then combine it with the linear SVC estimators into the pipeline `pipelinearsvm`: the `Pipeline` function simply takes a list of `scikit-learn` estimators and wraps them together into a new estimator object, which can then be passed to `GridSearchCV` via our `do_classify` function. Notice how this new estimator object can be used exactly the same way as a single classifier can be used in `scikit-learn`..this uniformity of interface is one of the nice features of `sklearn`!

In [34]:
selectorlinearsvm = SelectKBest(k=25, score_func=pearson_scorer)
pipelinearsvm = Pipeline([('select', selectorlinearsvm), ('svm', LinearSVC(loss="hinge"))])


#### Let us run the pipelined classifier 

We'll run the classifier and compare the results using the ROC curve to the previous SVM result.

In [35]:
pipelinearsvm, _,_,_,_  = do_classify(pipelinearsvm, {"svm__C": [0.00001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,lcols, u'result_14',1, reuse_split=reuse_split)

using reuse split
############# based on standard predict ################
Accuracy on training data: 0.58
Accuracy on test data:     0.56
[[123   0]
 [ 98   4]]
########################################################


What features did the pipelined classifier use? We can access them so:

In [36]:
np.array(lcols)[pipelinearsvm.get_params()['select'].get_support()]

array(['open', 'high', 'low', 'close', 'bb_upper', 'bb_middle', 'bb_lower',
       'kama_signal1', 'sar', 'plus_di', 'aroon_osc', 'macd',
       'macd_sigline', 'ppo', 'ppo_signal', 'mfi_signal', 'rsi', 'trix',
       'sr_pivotpts', 'sr_res1', 'sr_sup1', 'sr_res2', 'sr_sup2',
       'sr_res3', 'sr_sup3'], 
      dtype='|S13')

We plot the ROC curves, using the label `svm-feature-selected` for the pipelined classifier `pipelinearsvm`. We plot it alongside the older logistic-with lasso and all-features SVM for comparison

In [37]:
evaluate(pipelinearsvm)

accuracy on test set: 0.564
ROI baseline: 1.12%
ROI "result" buy-only: 32.48%
ROI "result" buy-sell: 72.07%
ROI "pred" buy-only: 5.69%
ROI "pred" buy-sell: 9.37%


### Balancing train set to test set for training.

In [38]:
jtrain=np.arange(0, ytrain.shape[0])
n_pos=len(jtrain[ytrain==1])
n_neg=len(jtrain[ytrain==0])
print n_pos, n_neg

716 542


There are many more negative samples in the training set. We want to balance the negative samples to the positive samples. So lets sample $n_{+}$ samples from the negative samples in the training set (without replacement).

In [40]:
ineg = np.random.choice(jtrain[ytrain==0], 500, replace=False)#n_pos, replace=False)

We concatenate all the indexes and use them to select a new training set from the old one.

In [41]:
alli=np.concatenate((jtrain[ytrain==1], ineg))
alli.shape

(1216,)

In [42]:
Xtrain_new = Xtrain[alli]
ytrain_new = ytrain[alli]
Xtrain_new.shape, ytrain_new.shape

((1216, 70), (1216,))

We store these into a new split variable `reuse_split_new`.

In [43]:
reuse_split_new=dict(Xtrain=Xtrain_new, Xtest=Xtest, ytrain=ytrain_new, ytest=ytest)

Note that the test sets are identical as before. This is as, even though we are training the SVM classifier in the "naturally" unfound situation of balanced classes, we **must test it in the real-world scenario of imbalance**.

#### 4.2 Train a linear SVM on this balanced set

Train a non-feature-selected linear SVM on this new balanced set as a comparison to both our old SVM on the imbalanced data set `clfsvm` and the feature-selected linear SVM `pipelinearsvm`. Store this new classifier in the variable `clfsvm_b`.

Compare the performances of all three of these classifiers using the roc curve plot, with the new `clfsvm_b` labeled as `svm-all-features-balanced`. 

In [44]:
#your code here
clfsvm_b = Pipeline([('select', selectorlinearsvm), ('svm', LinearSVC(loss="hinge"))])
clfsvm_b, _,_,_,_  = do_classify(clfsvm_b, {"svm__C": [0.00001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,lcols, u'result_14',1, reuse_split=reuse_split_new)

using reuse split
############# based on standard predict ################
Accuracy on training data: 0.59
Accuracy on test data:     0.56
[[123   0]
 [ 98   4]]
########################################################


In [45]:
clfsvm_b

Pipeline(steps=[('select', SelectKBest(k=25, score_func=<function pearson_scorer at 0x109b03410>)), ('svm', LinearSVC(C=1e-05, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

In [46]:
evaluate(clfsvm_b)

accuracy on test set: 0.564
ROI baseline: 1.12%
ROI "result" buy-only: 32.48%
ROI "result" buy-sell: 72.07%
ROI "pred" buy-only: 5.69%
ROI "pred" buy-sell: 9.37%


#### 4.3 Implement a RBF based pipelined (feature-selected) classifier on the balanced set.

In [47]:
from sklearn.svm import SVC

In [48]:
%%time
selectorsvm2 = SelectKBest(k=25, score_func=pearson_scorer)
pipesvm2 = Pipeline([('select2', selectorsvm2), ('svm2', SVC())])
pipesvm2, _,_,_,_  = do_classify(pipesvm2, {"svm2__C": [1e8, 1e9, 1e10], "svm2__gamma": [ 1e-9, 1e-10, 1e-11]}, dftouse,lcols, u'result_14',1, reuse_split=reuse_split_new)

using reuse split
############# based on standard predict ################
Accuracy on training data: 0.55
Accuracy on test data:     0.55
[[123   0]
 [102   0]]
########################################################
CPU times: user 3.92 s, sys: 41.8 ms, total: 3.96 s
Wall time: 5.24 s


In [49]:
pipesvm2.get_params()

{'select2': SelectKBest(k=25, score_func=<function pearson_scorer at 0x109b03410>),
 'select2__k': 25,
 'select2__score_func': <function __main__.pearson_scorer>,
 'svm2': SVC(C=100000000.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
   gamma=1e-09, kernel='rbf', max_iter=-1, probability=False,
   random_state=None, shrinking=True, tol=0.001, verbose=False),
 'svm2__C': 100000000.0,
 'svm2__cache_size': 200,
 'svm2__class_weight': None,
 'svm2__coef0': 0.0,
 'svm2__degree': 3,
 'svm2__gamma': 1e-09,
 'svm2__kernel': 'rbf',
 'svm2__max_iter': -1,
 'svm2__probability': False,
 'svm2__random_state': None,
 'svm2__shrinking': True,
 'svm2__tol': 0.001,
 'svm2__verbose': False}

In [50]:
evaluate(pipesvm2)

accuracy on test set: 0.547
ROI baseline: 1.12%
ROI "result" buy-only: 32.48%
ROI "result" buy-sell: 72.07%
ROI "pred" buy-only: 0.00%
ROI "pred" buy-sell: -1.12%
